In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

repo_name = {}

def read_data(filepath):
    d = pd.read_csv(filepath)
    return d

def clean_data():
    curr_millis = int(round(time.time()))

    d = read_data("watcher.csv")
    repo = d["repo_name"]
    watch_count = d["watch_count"]
    for i in range(len(repo)):
            repo_name[repo[i]] = [watch_count[i], np.nan, np.nan, np.nan, np.nan, np.nan]

    d = read_data("lang_count.csv")
    repo = d["L_repo_name"]
    lang_count = d["L_lang_count"]
    lan_sum = 0
    count = 0
    for i in range(len(repo)):
            if repo[i] in repo_name:
                    repo_name[repo[i]][1] = lang_count[i]
            if not pd.isnull(lang_count[i]):
                lan_sum += lang_count[i]
                count += 1
    
    mean_lang_count = lan_sum/count
    print(mean_lang_count)
    
    file_sum = 0
    count = 0
    d = read_data("files_count.csv")
    repo = d["S_repo_name"]
    file_count = d["C_files"]
    for i in range(len(repo)):
            if not pd.isnull(file_count[i]):
                    repo_name[repo[i]][2] = file_count[i]
            if not pd.isnull(file_count[i]):
                file_sum += file_count[i]
                count += 1
    
    mean_file_count = file_sum/count
    print(mean_file_count)

    commiter_sum = 0
    count = 0
    d = read_data("committer_count.csv")
    repo = d["S_repo_name"]
    committer_count = d["C_committer_count"]
    for i in range(len(repo)):
            if not pd.isnull(committer_count[i]):
                    repo_name[repo[i]][3] = committer_count[i]
            if not pd.isnull(committer_count[i]):
                commiter_sum += committer_count[i]
                count += 1
    
    mean_committer_count = commiter_sum/count
    print(mean_committer_count)

    commit_sum = 0
    count = 0
    d = read_data("commit_count.csv")
    repo = d["S_repo_name"]
    commit_count = d["C_comm_count"]
    for i in range(len(repo)):
            if not pd.isnull(commit_count[i]):
                    repo_name[repo[i]][4] = commit_count[i]
            if not pd.isnull(commit_count[i]):
                commit_sum += commit_count[i]
                count += 1
    
    mean_commit_count = commit_sum/count
    print(mean_commit_count)

    d = read_data("committer_date.csv")
    repo = d["S_repo_name"]
    commit_date = d["C_commit_date"]
    for i in range(len(repo)):
            if not pd.isnull(commit_date[i]):
                    dt_obj = datetime.strptime(commit_date[i][:-4],'%Y-%m-%d %H:%M:%S')
                    millisec = dt_obj.timestamp()
                    repo_name[repo[i]][5] = curr_millis - millisec
                    
    repo_name1 = repo_name.copy()
    print(len(repo_name1))
    for i in repo_name.keys():
            if pd.isnull(repo_name[i][5]):
                    del repo_name1[i]

    print(len(repo_name1))
    return (repo_name1, mean_lang_count, mean_file_count, mean_committer_count, mean_commit_count)

In [120]:
(repo_name1, mean_lang_count, mean_file_count, mean_committer_count, mean_commit_count) = clean_data()

2.5628183462
325.597920902
11.4284996691
849.54371336
400000
169216


In [121]:
df = pd.DataFrame.from_dict(repo_name1, orient='index')
df.columns = ["watch_count", "lang_count", "files_count", "committer_count", "commit_count", "commit_date"]

In [122]:
df.to_csv("Final.csv")

In [123]:
data = pd.read_csv("Final.csv")

In [127]:
LC = data['lang_count']
FC = data['files_count']
CC = data['committer_count']
COC = data['commit_count']
CD = data['commit_date']
WC = data['watch_count']
O = np.ones((len(FC),1))

#creating the x matix with the two features and excluding beta_0
X = np.zeros((len(FC),6))
X[:,0] = list(O)[:]
X[:,1] = LC[:]
X[:,2] = FC[:]
X[:,3] = CC[:]
X[:,4] = COC[:]
X[:,5] = CD[:]

null_val_index = np.argwhere(np.isnan(X))
for i in range(0,len(null_val_index)):
    if null_val_index[i][1] == 1:
        X[null_val_index[i][0]] = mean_lang_count
    elif null_val_index[i][1] == 2:
        X[null_val_index[i][0]] = mean_file_count
    elif null_val_index[i][1] == 3:  
        X[null_val_index[i][0]] = mean_committer_count
    elif null_val_index[i][1] == 4:
        X[null_val_index[i][0]] = mean_commit_count


Y = np.zeros((len(WC),1))
Y[:,0] = WC[:]

#Calculating Beta using Linear Regression ((X_T*X)^-1)(X_T*Y)
beta = np.matmul(np.linalg.inv(np.matmul(np.matrix.transpose(X),X)), np.matmul(np.matrix.transpose(X), Y))
beta.tolist()

[[-2.0401982802401264],
 [2.4611909525084417],
 [-0.001992407804806082],
 [1.134851732788944],
 [-0.0017559234229164188],
 [8.94242387419306e-08]]

In [128]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(X,Y)
model.coef_.tolist()

[[-2.9061833877340284,
  1.6774140931214483,
  -0.00196167954197147,
  1.160995342319948,
  -0.0016447942602627911,
  -1.0357489871991522e-07]]